# Task 4.8 Grouping Data & Aggregating Variables

## Contents
#### 1. Import Libraries & Dataframe
#### 
2. Find the aggregated mean of the “order_number” column grouped by “department_id
#### 3. Analyze the result
#### 4. Create a loyalty flag for existing customers using the transform() and loc() functions
#### 5. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category
#### 6. Create a spending flag for each user based on the average price across all their orders
#### 7. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column
#### Extra Data Cleaning”

## 1. Import Libraries & Dataframe

In [5]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [6]:
# Import dataframe
path = r'C:\\Users\\sstorer\\OneDrive\\Instacart Basket Analysis'
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_derived.pkl'))

## 2. Find the aggregated mean of the “order_number” column grouped by “department_id”

In [8]:
ords_prods_merge.groupby('department_id')['number_of_orders'].mean()

department_id
1     15.457838
2     17.277920
3     17.170395
4     17.811403
5     15.215751
6     16.439806
7     17.225802
8     15.340650
9     15.895474
10    20.197148
11    16.170638
12    15.887671
13    16.583536
14    16.773669
15    16.165037
16    17.665606
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: number_of_orders, dtype: float64

## 3. Analyze the result

These values represent the average number of orders per user for each department. Departments 21, 10, and 18 have the highest averages, meaning customers order from these departments more frequently. The values for the entire dataframe are very similar to those of the subset of the first million rows.

## 4. Create a loyalty flag for existing customers using the transform() and loc() functions

In [12]:
# Create a max_order column showing the maximum number of orders for each user
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['number_of_orders'].transform(np.max)

C:\Users\sstorer\AppData\Local\Temp\ipykernel_18304\99246066.py:2: FutureWarning: The provided callable <function max at 0x000002C839103CE0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['number_of_orders'].transform(np.max)


In [13]:
ords_prods_merge.shape

(32404859, 20)

In [14]:
ords_prods_merge.head()

,order_id,user_id,number_of_orders,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,merge_indicator,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,2539329,1,1,2,8,NaN,196,1,0,both,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10
1,2539329,1,1,2,8,NaN,14084,2,0,both,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10
2,2539329,1,1,2,8,NaN,12427,3,0,both,Original Beef Jerky,23,19,4.4,both,Low-range-product,Regularly busy,Regularly busy,Average orders,10
3,2539329,1,1,2,8,NaN,26088,4,0,both,Aged White Cheddar Popcorn,23,19,4.7,both,Low-range-product,Regularly busy,Regularly busy,Average orders,10
4,2539329,1,1,2,8,NaN,26405,5,0,both,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both,Low-range-product,Regularly busy,Regularly busy,Average orders,10


In [15]:
# Create a loyalty flag
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [16]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [17]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [18]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

## 5. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category

In [20]:
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

These values show the average, min, and max prices per item for each loyalty category. Loyal customers pay a lower price per item on average, and new customers pay the highest price per item on average.

## 6. Create a spending flag for each user based on the average price across all their orders

In [23]:
# Create a column showing the average price per item for each user
ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\sstorer\AppData\Local\Temp\ipykernel_18304\3534899658.py:2: FutureWarning: The provided callable <function mean at 0x000002C839118720> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [24]:
ords_prods_merge.head()

,order_id,user_id,number_of_orders,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,department_id,prices,merge_indicator,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price
0,2539329,1,1,2,8,NaN,196,1,0,both,...,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797
1,2539329,1,1,2,8,NaN,14084,2,0,both,...,16,12.5,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797
2,2539329,1,1,2,8,NaN,12427,3,0,both,...,19,4.4,both,Low-range-product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797
3,2539329,1,1,2,8,NaN,26088,4,0,both,...,19,4.7,both,Low-range-product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797
4,2539329,1,1,2,8,NaN,26405,5,0,both,...,17,1.0,both,Low-range-product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797


In [25]:
# Create a spending flag
ords_prods_merge.loc[ords_prods_merge['avg_price'] < 10, 'spending_flag'] = 'Low spender'

In [26]:
ords_prods_merge.loc[ords_prods_merge['avg_price'] >= 10, 'spending_flag'] = 'High spender'

In [27]:
ords_prods_merge.head()

,order_id,user_id,number_of_orders,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,prices,merge_indicator,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag
0,2539329,1,1,2,8,NaN,196,1,0,both,...,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender
1,2539329,1,1,2,8,NaN,14084,2,0,both,...,12.5,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender
2,2539329,1,1,2,8,NaN,12427,3,0,both,...,4.4,both,Low-range-product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender
3,2539329,1,1,2,8,NaN,26088,4,0,both,...,4.7,both,Low-range-product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender
4,2539329,1,1,2,8,NaN,26405,5,0,both,...,1.0,both,Low-range-product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender


In [28]:
ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     31770614
High spender      634245
Name: count, dtype: int64

## 7. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column

In [30]:
# Create a column showing the median days since prior order for each user
ords_prods_merge['median_days_since_prior_order'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

C:\Users\sstorer\AppData\Local\Temp\ipykernel_18304\939689060.py:2: FutureWarning: The provided callable <function median at 0x000002C839268360> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['median_days_since_prior_order'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [31]:
# Create an order frequency flag that marks regularity of a user's ordering behavior
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] > 20, 'order_frequency'] = 'Non-frequent customer'

In [32]:
ords_prods_merge.loc[(ords_prods_merge['median_days_since_prior_order'] > 10) & (ords_prods_merge['median_days_since_prior_order'] <= 20), 'order_frequency'] = 'Regular customer'

In [33]:
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] <= 10, 'order_frequency'] = 'Frequent customer'

In [34]:
ords_prods_merge.head(15)

,order_id,user_id,number_of_orders,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,median_days_since_prior_order,order_frequency
0,2539329,1,1,2,8,NaN,196,1,0,both,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2539329,1,1,2,8,NaN,14084,2,0,both,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,2539329,1,1,2,8,NaN,12427,3,0,both,...,Low-range-product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2539329,1,1,2,8,NaN,26088,4,0,both,...,Low-range-product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,2539329,1,1,2,8,NaN,26405,5,0,both,...,Low-range-product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
5,2398795,1,2,3,7,15.0,196,1,1,both,...,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
6,2398795,1,2,3,7,15.0,10258,2,0,both,...,Low-range-product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
7,2398795,1,2,3,7,15.0,12427,3,1,both,...,Low-range-product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
8,2398795,1,2,3,7,15.0,13176,4,0,both,...,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
9,2398795,1,2,3,7,15.0,26088,5,1,both,...,Low-range-product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [60]:
ords_prods_merge['order_frequency'].value_counts(dropna = False)

order_frequency
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: count, dtype: int64

## Extra Data Cleaning

In [63]:
# Find items with prices higher than 100 (outliers)
ords_prods_merge.loc[ords_prods_merge['prices'] > 100]

,order_id,user_id,number_of_orders,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,median_days_since_prior_order,order_frequency
1576,912404,17,12,2,14,5.0,21553,5,0,both,...,High-range product,Regularly busy,Regularly busy,Most orders,40,Regular customer,108.648299,High spender,5.0,Frequent customer
1638,603376,17,22,6,16,4.0,21553,3,1,both,...,High-range product,Regularly busy,Regularly busy,Most orders,40,Regular customer,108.648299,High spender,5.0,Frequent customer
16522,3264360,135,2,2,21,13.0,21553,6,0,both,...,High-range product,Regularly busy,Regularly busy,Average orders,4,New customer,1154.792308,High spender,12.0,Regular customer
16528,892534,135,3,0,8,12.0,21553,3,1,both,...,High-range product,Busiest day,Busiest days,Average orders,4,New customer,1154.792308,High spender,12.0,Regular customer
53672,229704,342,8,1,19,30.0,21553,9,0,both,...,High-range product,Regularly busy,Busiest days,Average orders,16,Regular customer,114.426619,High spender,23.0,Non-frequent customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32320511,3172853,205650,18,1,9,7.0,21553,17,1,both,...,High-range product,Regularly busy,Busiest days,Most orders,25,Regular customer,351.141618,High spender,6.0,Frequent customer
32347717,2504315,205818,3,5,15,3.0,21553,13,0,both,...,High-range product,Regularly busy,Regularly busy,Most orders,25,Regular customer,170.073770,High spender,13.0,Regular customer
32347727,1108388,205818,5,4,5,1.0,21553,5,1,both,...,High-range product,Least busy,Slowest days,Fewest orders,25,Regular customer,170.073770,High spender,13.0,Regular customer
32380527,1916142,206049,1,2,17,NaN,21553,2,0,both,...,High-range product,Regularly busy,Regularly busy,Average orders,5,New customer,938.031250,High spender,7.0,Frequent customer


In [65]:
# Assign Nan values to outliers
ords_prods_merge.loc[ords_prods_merge['prices'] >100, 'prices'] = np.nan

In [67]:
ords_prods_merge['prices'].max()

25.0

In [69]:
# Drop merge flag columns
ords_prods_merge = ords_prods_merge.drop('_merge', axis=1)
ords_prods_merge = ords_prods_merge.drop('merge_indicator', axis = 1)

In [71]:
# Set to display all columns
pd.set_option('display.max_columns', None)

In [73]:
ords_prods_merge.head()

,order_id,user_id,number_of_orders,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,median_days_since_prior_order,order_frequency
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2539329,1,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,2539329,1,1,2,8,NaN,12427,3,0,Original Beef Jerky,23,19,4.4,Low-range-product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2539329,1,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,Low-range-product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,2539329,1,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,Low-range-product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [75]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_merge_grouped.pkl'))